In [ ]:
######################
# Auto-PTA processor #
######################
#
# Post-processing for the AutoPTA output data:
# - Clips well data as per global dictionary (e.g. for failed gauges, changing derivative profiles)
# - Merges clipped well data if we have many>one well mapping
# - Checks preferred shut-in ID status and fixes ID number if necessary (e.g. update to time range, detection parameters)
# - Uses preferred shut-in ID to perform QC scoring on each PBU/PFO based on covariance correlation to trusted data
# - Outputs processed plots/diagnostic dfs in addition to QC score df (Spotfire does the rest)
#
# Last updated: 08/11/19, v1.0
# Contact: D Robbins
#
#
# Global libraries used
import numpy as np
import pandas as pd
from scipy.stats import spearmanr
#
###################################
# Global variables and thresholds #
###################################
#
f_unique_id_threshold = 12.0 # number of hours to be acceptable for unique ID start point to have moved
#
################
# Global lists #
################
#
# list of wells to remove from df (usually just use the clip/merge dictionaries below; may have older wells/bores no longer used)
l_remove_well = ['I01'] # removed CW19 as this is the old well (CW19z), replaced by CW19x side track
#
#######################
# Global dictionaries #
#######################
#
# Wells to clip/merge 
#
# Clip dictionary: well:[start,end]
i_clipdata_clipfrom = 0
i_clipdata_clipto = 1
#
d_clip_data = {
    'I02_A':['01/01/2000 00:00:00','01/03/2008 00:00:00'],
    'I02_B':['10/06/2012 00:00:00','01/01/2100 00:00:00'],
    }
#
# Merge dictionary for clipped wells: global_well:[clipped_well_1,clipped_well_2,...,clipped_well_n]
d_merge_wells = {
    'I02':['I02_A','I02_B'],
    }
#
# Logarithmic sampling dictionary (sets up the np.logspace sampling for reference vs non-reference covariance calculation)
#
# List elements for each dictionary entry:
i_logsample_min = 0         # log10 start point on derivative plot (e.g. -1 = 10^-1 = 0.1)
i_logsample_max = 1         # log10 end point on derivative plot (e.g. 2 = 10^2 = 100) - use shorter end points for known offset interference
i_logsample_samples = 2     # samples per log cycle (recommend 30 - 50)
i_logsample_method = 3      # QC score method from 2 closest neighbours; 0 = maximum (recommended), 1 = closest, 2 = distance-weighted average
i_logsample_kh_penalty = 4  # apply a (log kh ratio / n) penalty to QC score; 0.0 = no penalty, recommend 1.0-2.0
#
d_logsample_params = {
    #
    'I01':[-1,3,40,0,1.25],
    'I02':[-1,3,40,0,1.25],
    }
#
####################
# Global functions #
####################
#
# Function to find the nearest two neighbours to a value (you don't need to do anything to this)
def nearest_two_neighbours(value,lst):
    #
    my_two_values = []
    #
    # build list of distances
    my_distance = [(i-value) for i in lst]
    my_euclid_distance = [abs(i-value) for i in lst]
    #
    # spit out first value (remember to use the sign not absolute Euclidean!)
    my_two_values.append(my_distance[my_euclid_distance.index(min(my_euclid_distance))]+value)
    #
    # remove first value from two distance lists and any others on the same side
    if (value - (my_distance[my_euclid_distance.index(min(my_euclid_distance))]+value)) < 0: # remove greater than
        new_lst = [i for i in lst if i < my_distance[my_euclid_distance.index(min(my_euclid_distance))]+value]
    else:
        new_lst = [i for i in lst if i > my_distance[my_euclid_distance.index(min(my_euclid_distance))]+value]
    #
    # re-build list of distances
    my_distance = [(i-value) for i in new_lst]
    my_euclid_distance = [abs(i-value) for i in new_lst]
    #
    # return the second value (use try-except if we only have one side)
    try:
        my_two_values.append(my_distance[my_euclid_distance.index(min(my_euclid_distance))]+value)
    except:
        pass
    #
    # return two nearest neighbours
    return my_two_values
#

In [ ]:
def process_plots(plots):
    #
    # first point of processing for the PLOTS data table (derivative/dP data):
    # - perform datetime clips on wells as pre requested
    # - merge multiple pseudo-wells
    #
    df = plots.copy(deep=True)
    #
    # update datetime column to make it TZ-aware
    df['start_time'] = pd.to_datetime(df['start_time'],utc=True)
    #
    # build list of wells to leave alone, refer to global dictionaries
    my_keep_wells = []
    for well in df['well_id'].unique():
        if well not in l_remove_well and well not in d_clip_data:
            my_keep_wells.append(well)
    #
    # split the df into wells to clip and wells to leave alone
    df_toclip = df[df['well_id'].isin(d_clip_data)]
    df = df[df['well_id'].isin(my_keep_wells)]
    #
    # apply the clipping requested in the global dictionary
    df_clip = pd.DataFrame(columns=df.columns)
    #
    for well in df_toclip['well_id'].unique():
        #
        # pull clips from the dictionary
        clip_from = pd.to_datetime(d_clip_data[well][i_clipdata_clipfrom],utc=True,dayfirst=True)
        clip_to = pd.to_datetime(d_clip_data[well][i_clipdata_clipto],utc=True,dayfirst=True)
        #
        # build df of clipped data for this well
        df_well = df_toclip[df_toclip['well_id']==well]
        #
        # perform clip
        df_well = df_well[(df_well['start_time']>=clip_from)&(df_well['start_time']<=clip_to)]
        #
        # add to df of clipped data
        df_clip = pd.concat([df_clip,df_well],ignore_index=True)
        #
    #
    # now merge the wells in the clipped df
    df_merge = pd.DataFrame(columns=df.columns)
    #
    for well in d_merge_wells:
        #
        # pull list of labels to merge
        l_to_merge = d_merge_wells[well]
        #
        # verify that merge wells exist in df
        for merge_well in l_to_merge:
            if len(df_clip[df_clip['well_id']==merge_well]) == 0:
                print('Warning: well ',merge_well," doesn't exist in clip dataframe (d_merge_cells,d_clip_data)")
        #
        # rename the column names in the clip columns to be the new unified well
        df_well = df_clip[df_clip['well_id'].isin(l_to_merge)].copy(deep=True)
        df_well.loc[:,'well_id'] = [well]*len(df_well)
        df_well = df_well.sort_values(by='start_time')
        #
        # add to df of merged data
        df_merge = pd.concat([df_merge,df_well],ignore_index=True)
        #
    #
    # merge df with the master df
    df = pd.concat([df,df_merge],ignore_index=True)
    #
    # replace blanks with NaN so PySpark doesn't fall over
    df.replace('',np.nan,inplace=True)
    #
    print('Successfully clipped/merged plots data table.')
    #
    return df

In [ ]:
def process_PTA(pta):
    #
    # first point of processing for the PLOTS data table (derivative/dP data):
    # - perform datetime clips on wells as pre requested
    # - merge multiple pseudo-wells
    #
    df = pta.copy(deep=True)
    #
    # update datetime column to make it TZ-aware
    df['start_time'] = pd.to_datetime(df['start_time'],utc=True)
    #
    # build list of wells to leave alone, refer to global dictionaries
    my_keep_wells = []
    for well in df['well_id'].unique():
        if well not in l_remove_well and well not in d_clip_data:
            my_keep_wells.append(well)
    #
    # split the df into wells to clip and wells to leave alone
    df_toclip = df[df['well_id'].isin(d_clip_data)]
    df = df[df['well_id'].isin(my_keep_wells)]
    #
    # apply the clipping requested in the global dictionary
    df_clip = pd.DataFrame(columns=df.columns)
    #
    for well in df_toclip['well_id'].unique():
        #
        # pull clips from the dictionary
        clip_from = pd.to_datetime(d_clip_data[well][i_clipdata_clipfrom],utc=True,dayfirst=True)
        clip_to = pd.to_datetime(d_clip_data[well][i_clipdata_clipto],utc=True,dayfirst=True)
        #
        # build df of clipped data for this well
        df_well = df_toclip[df_toclip['well_id']==well]
        #
        # perform clip
        df_well = df_well[(df_well['start_time']>=clip_from)&(df_well['start_time']<=clip_to)]
        #
        # add to df of clipped data
        df_clip = pd.concat([df_clip,df_well],ignore_index=True)
        #
    #
    # now merge the wells in the clipped df
    df_merge = pd.DataFrame(columns=df.columns)
    #
    for well in d_merge_wells:
        #
        # pull list of labels to merge
        l_to_merge = d_merge_wells[well]
        #
        # verify that merge wells exist in df
        for merge_well in l_to_merge:
            if len(df_clip[df_clip['well_id']==merge_well]) == 0:
                print('Warning: well ',merge_well," doesn't exist in clip dataframe (d_merge_cells,d_clip_data)")
        #
        # rename the column names in the clip columns to be the new unified well
        df_well = df_clip[df_clip['well_id'].isin(l_to_merge)].copy(deep=True)
        df_well.loc[:,'well_id'] = [well]*len(df_well)
        df_well = df_well.sort_values(by='start_time')
        #
        # add to df of merged data
        df_merge = pd.concat([df_merge,df_well],ignore_index=True)
        #
    #
    # merge df with the master df
    df = pd.concat([df,df_merge],ignore_index=True)
    #
    # replace blanks with NaN so PySpark doesn't fall over
    df.replace('',np.nan,inplace=True)
    #
    print('Successfully clipped/merged PTA data table.')
    #
    return df

In [ ]:
def reset_shutin_ids(pta):
    #
    # reset shut-in IDs and build df of the mapping performed
    #
    # first, we build a list of the shut-in IDs from the original PTA df
    # (there's probably a more efficient way to do this, but I did all the mods at the same time
    #    in my Jupyter notebook but I can't output many dfs in one Code Workbook transform ...)
    #
    # original PTA df
    df = pta.copy(deep=True)
    #
    # update datetime column to make it TZ-aware
    df['start_time'] = pd.to_datetime(df['start_time'],utc=True)
    #
    # split the df into wells to clip and wells to leave alone
    df_toclip = df[df['well_id'].isin(d_clip_data)]
    #
    # apply the clipping requested in the global dictionary
    df_clip = pd.DataFrame(columns=df.columns)
    #
    for well in df_toclip['well_id'].unique():
        #
        # pull clips from the dictionary
        clip_from = pd.to_datetime(d_clip_data[well][i_clipdata_clipfrom],utc=True,dayfirst=True)
        clip_to = pd.to_datetime(d_clip_data[well][i_clipdata_clipto],utc=True,dayfirst=True)
        #
        # build df of clipped data for this well
        df_well = df_toclip[df_toclip['well_id']==well]
        #
        # perform clip
        df_well = df_well[(df_well['start_time']>=clip_from)&(df_well['start_time']<=clip_to)]
        #
        # add to df of clipped data
        df_clip = pd.concat([df_clip,df_well],ignore_index=True)
        #
    #
    # with the clipped df, build a list (of lists) of well - old shut-in ID - new shut-in ID
    #
    l_shutin_convert = []
    #
    for well in d_merge_wells:
        #
        # build list of clipped unique shut-in IDs and re-assign
        l_well_si_ids = df_clip[df_clip['well_id'].isin(d_merge_wells[well])][['start_time','shutin_id']].sort_values(by='start_time')
        l_well_si_ids = l_well_si_ids['shutin_id'].tolist()
        #
        # loop over list and generate mapping of old>new shut-in ID
        for i_si,shut_in in enumerate(l_well_si_ids):
            new_shutin_id = str('GL_')+str(well)+str('_')+str(i_si+1)
            l_shutin_convert.append([well,shut_in,new_shutin_id])
        #
    #
    # write list-of-lists to df
    df = pd.DataFrame(l_shutin_convert,columns=['well_id','old_shutin_id','new_shutin_id'])
    #
    print('Built merged/spliced well shut-in ID conversion OK.')
    #
    return df

In [ ]:
def reset_PTA_ids(process_PTA, reset_shutin_ids):
    #
    # take the processed (clipped/merged) plots df and assign new shut-in IDs
    # - this only happens on clipped/merged wells, so just loop over these
    #
    # copy the two dataframes
    df = process_PTA.copy(deep=True)
    df_shutins = reset_shutin_ids.copy(deep=True)
    #
    for well in d_merge_wells:
        #
        # generate shut-in df, use the old shut-in ID as the index so we can df.loc on this
        df_shutin_well = df_shutins[df_shutins['well_id']==well]
        #
        # build dictionary from the shut-in IDs
        d_well_shutins = {}
        for i_df,row_df in df_shutin_well.iterrows():
            d_well_shutins[row_df['old_shutin_id']] = row_df['new_shutin_id']
        #
        # loop over shut-ins and re-assign shut-in ID
        for shutin_id in df[df['well_id']==well]['shutin_id'].unique():
            try:
                df.loc[df['shutin_id']==shutin_id,'shutin_id'] = d_well_shutins[shutin_id]
            except:
                pass
    #
    print('Successfully re-assigned shut-in IDs in the PTA data table.')
    #
    return df  

In [ ]:
def reset_plots_IDs(process_plots, reset_shutin_ids, reset_PTA_ids):
    #
    # take the processed (clipped/merged) plots df and assign new shut-in IDs
    # - this only happens on clipped/merged wells, so just loop over these
    #
    # for the plots df, we also unify shut-in IDs on the PTA df to ensure consistency
    #
    # copy the two dataframes
    df = process_plots.copy(deep=True)
    df_shutins = reset_shutin_ids.copy(deep=True)
    df_PTA = reset_PTA_ids.copy(deep=True)
    #
    for well in d_merge_wells:
        #
        # generate shut-in df, use the old shut-in ID as the index so we can df.loc on this
        df_shutin_well = df_shutins[df_shutins['well_id']==well]
        #
        # build dictionary from the shut-in IDs
        d_well_shutins = {}
        for i_df,row_df in df_shutin_well.iterrows():
            d_well_shutins[row_df['old_shutin_id']] = row_df['new_shutin_id']
        #
        # loop over shut-ins and re-assign shut-in ID
        for shutin_id in df[df['well_id']==well]['shutin_id'].unique():
            try:
                df.loc[df['shutin_id']==shutin_id,'shutin_id'] = d_well_shutins[shutin_id]
            except:
                pass
    #
    # clip the plots df on the basis of shut-in IDs identified in the main PTA df
    #
    i_len_df = len(df)                                          # remember the length pre-clip
    df = df[df['shutin_id'].isin(df_PTA['shutin_id'].unique())] # perform the clip
    #
    print('Successfully re-assigned shut-in IDs in the plots data table.')
    print('Plots dataframe size is: %.2f pc of original after unifying shut-in IDs.')%((len(df)/float(i_len_df))*100.0)
    #
    return df  

In [ ]:
def update_reference_shutin_ids(reset_PTA_ids, preferred_shutin_ids):
    #
    # Loop through the preferred shut-ins selected and determine if they are still the same
    # -> if ID has changed, update the unique shut-in ID
    #
    # Initialise loop counters
    i_matched = 0
    i_different_ID = 0
    i_slight_shift = 0
    i_slight_shift_newID = 0
    i_failed = 0
    my_new_preferred_list = []
    #
    print('Processing unique ID data ...')
    #
    # deep copy input dataframes
    df_PTA = reset_PTA_ids.copy(deep=True)
    df_preferred_shutins = preferred_shutin_ids.copy(deep=True)
    #
    # set up date-time format on preferred shut-ins df column
    df_preferred_shutins['pbu_pfo_date'] = pd.to_datetime(df_preferred_shutins['pbu_pfo_date'],utc=True,dayfirst=True)
    df_PTA['start_time'] = pd.to_datetime(df_PTA['start_time'],utc=True)
    #
    # loop over preferred shut-in ID df
    for i,row in df_preferred_shutins.iterrows():
        #
        # initialise shut-in ID status
        i_check_QC = 0 # if this becomes 1, we believe shut-in ID hasn't been changed or we have found a match
        #
        # grab row information
        shutin_well = row['well_id']
        shutin_date = row['pbu_pfo_date']
        old_shutin_id = row['shutin_id']
        #
        # look up this shut in ID on the new data (will create empty series if no slice found)
        s_new_data = df_PTA[df_PTA['shutin_id']==old_shutin_id]
        t_new_data = df_PTA[(pd.to_datetime(df_PTA['start_time'])==shutin_date) & (df_PTA['well_id']==shutin_well)]
        #   
        # if we find a PBU with exactly the same date-time, take this point as the new unique ID
        if len(t_new_data) == 1:
            i_check_QC = 1
            i_store_ID = t_new_data['shutin_id'].iloc[0]
            date_store = shutin_date
            if i_store_ID == old_shutin_id:
                i_matched += 1 # counter of exactly matched unique IDs
            else:
                i_different_ID += 1 # counter of changed unique IDs at same date-time
        else:
            # if it's not exactly matched, firstly check unique ID match if there is one
            if len(s_new_data) == 1: # shouldn't be more than 1 row, but just in case don't use >0 logic
                #
                # check the time here
                new_ID_time = s_new_data['start_time'].iloc[0]
                delta_t = abs(new_ID_time - shutin_date)/pd.Timedelta(1,'h')
                #
                # if the time delta is "close enough" then use this shutin ID (threshold value is set in global code)
                if delta_t < f_unique_id_threshold:
                    i_check_QC = 1
                    i_store_ID = s_new_data['shutin_id'].iloc[0]
                    i_slight_shift += 1
                    date_store = new_ID_time
            #
            # if no (or multiple?!) unique ID match(es), look to find a PBU/PFO within a slight time offset
            # also check here if we find that the unique ID doesn't match the other one
            if i_check_QC != 1:
                #
                # grab new df using well and shut-in data += a timedelta offset (use 12 hrs as a guess)
                t_new_data = df_PTA[(df_PTA['well_id']==shutin_well)&(df_PTA['start_time']<(shutin_date+pd.Timedelta(f_unique_id_threshold,'h')))&(df_PTA['start_time']>(shutin_date-pd.Timedelta(f_unique_id_threshold,'h')))]
                #
                # if only one PBU/PFO found within timedelta window, use this as the updated reference shut-in ID
                if len(t_new_data) == 1:
                    i_check_QC = 1
                    i_store_ID = t_new_data['shutin_id'].iloc[0]
                    i_slight_shift_newID += 1
                    date_store = t_new_data['start_time'].iloc[0]
                # if more than one, find the closest
                elif len(t_new_data) > 1:
                    t_delta = 100000.0
                    date_store = t_new_data['start_time'].iloc[0] # placeholder just in case, will be overwritten in loop
                    # loop over each shut-in in subset timedelta window and find the closest to the original shut-in time
                    for i,row in t_new_data.iterrows():
                        delta_t = abs(row['start_time'] - shutin_date)/pd.Timedelta(1,'h')
                        if delta_t < t_delta:
                            t_delta = delta_t
                            date_store = row['start_time']
                            my_shutin_ID_loop = row['shutin_id']  
                    i_check_QC = 1
                    i_store_ID = my_shutin_ID_loop
                    i_slight_shift_newID += 1
                else:
                    # report that we couldn't find a PBU/PFO close to the data
                    i_check_QC = 2
                    i_failed += 1
                    print('Failed to match old ID: ',old_shutin_id)
                #
            #
        #
        # build the new preferred shut-in df
        if i_check_QC == 1:
            my_new_preferred_list.append([shutin_well,i_store_ID,'Y',date_store])
        #
        # print some debug information
        if i_check_QC == 0:
            print('WARNING: Unique ID ',old_shutin_id,' failed to find a match and unexpectedly passed through all the logic ... fail')
    #
    # write the list of preferred shut-ins to new df
    df_preferred_shutins_new = pd.DataFrame(my_new_preferred_list,columns=['well_id','shutin_id','shutin_preferred','pbu_pfo_date'])
    #
    # print some summary statistics
    print('')
    print(str('Number of unique PFOs in old file: '+str(len(df_preferred_shutins))))
    print(str('Number of unique PFOs matched: '+str(i_matched+i_different_ID+i_slight_shift+i_slight_shift_newID)))
    print(str('... exactly matched on date-time, same ID: '+str(i_matched)))
    print(str('... exactly matched on date-time, different ID: '+str(i_different_ID)))
    print(str('... slightly shifted, same ID: '+str(i_slight_shift)))
    print(str('... slightly shifted, different ID: '+str(i_slight_shift_newID)))
    print(str('Failed to match: '+str(i_failed)))
    #
    # return df
    return df_preferred_shutins_new

In [ ]:
def qc_analysis(update_reference_shutin_ids, reset_plots_IDs):
    #
    # Perform QC analysis of all shut-in events
    # -> computes Spearman rank coefficient between shut-in and reference shut-in
    # -> computes penalty coefficient for kh ratio offset
    #
    # this is disgustingly inefficient but works ... 
    #
    # make deep copy of each df
    df_plots = reset_plots_IDs.copy(deep=True)
    df_preferred_shutins_new = update_reference_shutin_ids.copy(deep=True)
    #
    # firstly, set up a new "unique_id" column that we'll use to derive closest neighbour in a fast manner
    df_plots['id_number'] = df_plots['shutin_id'].str.split("_").str[-1]
    df_preferred_shutins_new['id_number'] = df_preferred_shutins_new['shutin_id'].str.split("_").str[-1]
    #
    # convert split numbers to integers
    df_plots = df_plots.astype({'id_number':'int64'})
    df_preferred_shutins_new = df_preferred_shutins_new.astype({'id_number':'int64'})
    #
    # set up QC lists
    my_QC_id = []
    my_QC_score = []
    my_kh_ratio = []
    #
    # get number of wells to generate an excruciating counter
    i_no_wells = len(df_plots['well_id'].unique())
    #
    # loop over all wells and perform QC
    i_wells_QC = 0 # counter
    #
    for well in df_plots['well_id'].unique():
        #
        # check that this well has preferred shut-in ids
        df_well_pref = df_preferred_shutins_new[df_preferred_shutins_new['well_id']==well]
        #
        # get the df for this well
        df_well = df_plots[df_plots['well_id']==well]
        #
        # get the sampling parameters (from global dictionary)
        try:
            l_logsample_params = d_logsample_params[well]
        except: # fall back option is CP01 parameters, as CP01 is the best well
            l_logsample_params = d_logsample_params['CP01']
        #
        # if the well does have preferred shut-ins identified, loop over all shut-ins and do the QC scoring
        if len(df_well_pref) > 0:
            #
            # list of reference PBU/PFOs
            l_ref_shutin = df_well_pref['shutin_id'].tolist()
            #
            # loop over each PBU/PFO (I call the variable PFO because I'm on team water injection)
            for PFO in df_well['shutin_id'].unique():
                #
                # if shut-in is a preferred one, set QC = 2 and move on
                if PFO in l_ref_shutin:
                    #
                    # set up QC = 2 if well is a reference shut-in
                    my_QC = 2.0
                    #
                    my_QC_id.append(PFO)
                    my_QC_score.append(my_QC)
                    my_kh_ratio.append(1.0)
                    #
                #
                # if shut-in isn't a preferred one, perform QC scoring
                else:
                    #
                    # get PFO dataframe (only use derivative plot, not dP, to do the QC as dP position is a function of skin)
                    df_PFO = df_well[(df_well['shutin_id']==PFO)&(df_well['series']=='Derv')]
                    #
                    # find closest reference shut-in; may only have one if all on same side
                    # (nearest two neighbours is a global function)
                    n2n = nearest_two_neighbours(df_PFO['id_number'].iloc[0],df_well_pref['id_number'].tolist())
                    #
                    # loop over the two closest reference PFOs and calculate the Spearman rank coefficient
                    #
                    # (lists to build for spearman rank coefficient, timedelta to ref PFO, kh/stabilisation observed)
                    ref_S_list = []
                    ref_S_dt = []
                    ref_kh_list = []
                    #
                    for ref_pfo in n2n:
                        #
                        # pull the reference PFO (df_PFO_ref) to compare the shut-in (df_PFO) against
                        df_PFO_ref = df_well[(df_well['shutin_id']==('GL_'+well+'_'+str(ref_pfo)))&(df_well['series']=='Derv')]
                        #
                        # proceed if dfs have been pulled properly
                        if (len(df_PFO) > 0) and (len(df_PFO_ref) > 0):
                            #
                            # build the log sampling grid (i_logsample_X is the integer reference to the list supplied in the d_logsample_params dictionary)
                            sample_grid = np.logspace(l_logsample_params[i_logsample_min],l_logsample_params[i_logsample_max],num=int(l_logsample_params[i_logsample_samples]*(l_logsample_params[i_logsample_max]-l_logsample_params[i_logsample_min])))
                            #
                            # reset derivative lists
                            ave_value_compare = []
                            ave_value_ref = []
                            #
                            # build the logarithmic grid sampled derivatives
                            for i in range(0,len(sample_grid)-1):
                                #
                                filtered_data = df_PFO[(df_PFO['delta_time']>=sample_grid[i])&(df_PFO['delta_time']<sample_grid[i+1])]['value']
                                filtered_data_ref = df_PFO_ref[(df_PFO_ref['delta_time']>=sample_grid[i])&(df_PFO_ref['delta_time']<sample_grid[i+1])]['value']
                                #
                                # incorporate this sampled bucket ONLY if both PFOs have values
                                if (len(filtered_data) > 0) & (len(filtered_data_ref) > 0):
                                    ave_value_compare.append(np.mean(filtered_data))
                                    ave_value_ref.append(np.mean(filtered_data_ref))
                                #
                            #
                            # generate the covariance coefficient if more than one value in list, otherwise set value to zero (no QC)
                            if (len(ave_value_compare) > 1) & (len(ave_value_ref) > 1):
                                ref_S_list.append(spearmanr(ave_value_compare,ave_value_ref)[0])
                            else:
                                ref_S_list.append(0.0)
                            #
                            # append the delta t (for weighting between two reference PFOs if applicable)
                            ref_S_dt.append(abs((df_PFO_ref['start_time'].iloc[0] - df_PFO['start_time'].iloc[0]).total_seconds()))
                            #
                            # append the kh ratio of the two derivative plots
                            ref_kh_list.append(df_PFO_ref['stabilisation_value'].iloc[0]/df_PFO['stabilisation_value'].iloc[0])
                        #
                        # if no data have been pulled for this reference PFO, append a fat zero ...
                        else:
                            ref_S_list.append(0.0)
                            ref_S_dt.append(1000000000)
                            ref_kh_list.append(1.0)
                    #
                    # generate the QC score averaged/sampled across the two nearest reference PFOs
                    #
                    # options specified in the global dictionary:
                    # 0 = maximum (recommended), 1 = closest, 2 = distance-weighted average
                    #
                    # option 1 - use the maximum QC score (recommended)
                    if l_logsample_params[i_logsample_method] == 0:
                        my_QC = max(ref_S_list)
                        my_kh = ref_kh_list[ref_S_list.index(max(ref_S_list))]
                    #
                    # option 2 - use the closest QC score
                    elif l_logsample_params[i_logsample_method] == 1:
                        if len(ref_S_list) > 1:
                            my_QC = 0.0
                            my_kh = 1.0
                            loop_dt = 0.0
                            for S,dt,kh in zip(ref_S_list,ref_S_dt,ref_kh_list):
                                if dt > loop_dt:
                                    loop_dt = dt
                                    my_QC = S
                                    my_kh = kh
                        else:
                            my_QC = ref_S_list[0]
                            my_kh = ref_kh_list[0]
                    #
                    # option 3 - weight by (time) distance if more than one value
                    else: 
                        if len(ref_S_list) > 1:
                            my_QC = 0.0
                            my_kh = 0.0
                            for S,dt,kh in zip(ref_S_list,ref_S_dt,ref_kh_list):
                                my_QC += S*(1.0-(dt/sum(ref_S_dt)))
                                my_kh += kh*(1.0-(dt/sum(ref_S_dt)))
                        else:
                            my_QC = ref_S_list[0]
                            my_kh = ref_kh_list[0]
                    #
                    # finally, apply kh penalty to QC score
                    if l_logsample_params[i_logsample_kh_penalty] > 0.0:
                        try:
                            # apply penalty to QC score
                            my_QC -= abs(np.log10(my_kh))/l_logsample_params[i_logsample_kh_penalty]
                            # apply thresholds to ensure QC bound between [-1.0, 1.0]
                            my_QC = min(1.0,max(-1.0,my_QC))
                            #
                        except: # no kh score / NaN error
                            my_QC = -0.99 # rather than -1.0 so should enable us to see them in SPF ...
                    #
                    # write S scores and kh ratio to list
                    my_QC_id.append(PFO)
                    my_QC_score.append(my_QC)
                    my_kh_ratio.append(my_kh)
                #
            # end of loop over each PFO in well
            print('Well '+well+' QC performed successfully.')
        #
        # for all wells with NO reference derivatives, append a zero score by default
        else:
            for PFO in df_well['shutin_id'].unique():
                my_QC_id.append(PFO)
                my_QC_score.append(0.0)
                my_kh_ratio.append(1.0)
            print('No QC performed for well '+well+' - no reference derivatives found.')
        #
        i_wells_QC += 1
        #
        print(str('QC completion status: ')+str(i_wells_QC)+str('/')+str(i_no_wells))
    #
    # generate QC df from the lists
    df_QC = pd.DataFrame()
    df_QC.insert(len(df_QC.columns),'shutin_id',my_QC_id)
    df_QC.insert(len(df_QC.columns),'qc_score',my_QC_score)
    df_QC.insert(len(df_QC.columns),'kh_ratio',my_kh_ratio)
    #
    # return the df
    return df_QC